In [35]:
import os
file_names:list[str] = os.listdir('stations')
abs_paths:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_paths.append(os.path.abspath(f'stations/{name}'))
    
abs_paths

['/workspaces/2024-2-22/lesson22/stations/每日各站進出站人數2020.csv',
 '/workspaces/2024-2-22/lesson22/stations/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/2024-2-22/lesson22/stations/每日各站進出站人數2023.csv',
 '/workspaces/2024-2-22/lesson22/stations/每日各站進出站人數2021.csv',
 '/workspaces/2024-2-22/lesson22/stations/每日各站進出站人數2022.csv']

In [36]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_paths]
display(all_df)


[       trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20200101      900             7552             8154
 1       20200101      910             1020             1135
 2       20200101      920             1623             1972
 3       20200101      930             4130             4813
 4       20200101      940             1818             2128
 ...          ...      ...              ...              ...
 86927   20201231     7360             3523             3602
 86928   20201231     7361                3               18
 86929   20201231     7362               28               34
 86930   20201231     7380              609              604
 86931   20201231     7390              377              444
 
 [86932 rows x 4 columns],
        trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20190423      900             8442             7743
 1       20190423      910             1394             1348
 2       20190423      920             2770             

In [37]:
inOut_df = pd.concat(all_df)
inOut_df.columns = ['乘車日','車站代碼','進站人數','出站人數']
inOut_df

,乘車日,車站代碼,進站人數,出站人數
0,20200101,900,7552,8154
1,20200101,910,1020,1135
2,20200101,920,1623,1972
3,20200101,930,4130,4813
4,20200101,940,1818,2128
...,...,...,...,...
86547,20221231,7360,3984,3893
86548,20221231,7361,92,115
86549,20221231,7362,131,159
86550,20221231,7380,494,426


In [38]:
import requests
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

,車站代碼,中文站名
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [39]:
station_all_df = pd.merge(inOut_df,station_df,on='車站代碼',how='left')
station_all_df

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20200101,900,7552,8154,基隆
1,20200101,910,1020,1135,三坑
2,20200101,920,1623,1972,八堵
3,20200101,930,4130,4813,七堵
4,20200101,940,1818,2128,百福
...,...,...,...,...,...
406756,20221231,7360,3984,3893,瑞芳
406757,20221231,7361,92,115,海科館
406758,20221231,7362,131,159,八斗子
406759,20221231,7380,494,426,四腳亭


In [40]:
station_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [41]:
#移除Na
station_all_df1 = station_all_df.dropna()
station_all_df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [42]:
#乘車日變為datetime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str))
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站代碼    406761 non-null  int64         
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
 4   中文站名    406761 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 15.5+ MB


In [43]:
station_all_df1

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,2020-01-01,900,7552,8154,基隆
1,2020-01-01,910,1020,1135,三坑
2,2020-01-01,920,1623,1972,八堵
3,2020-01-01,930,4130,4813,七堵
4,2020-01-01,940,1818,2128,百福
...,...,...,...,...,...
406756,2022-12-31,7360,3984,3893,瑞芳
406757,2022-12-31,7361,92,115,海科館
406758,2022-12-31,7362,131,159,八斗子
406759,2022-12-31,7380,494,426,四腳亭


In [44]:
station_all_df1.groupby('中文站名')[['進站人數','出站人數']].agg(['sum','max','min'])

進站人數                出站人數           
          sum   max  min      sum   max  min
中文站名                                        
七堵    9044563  7704  141  9115329  8288  210
三坑    2080517  1715   34  2140894  1835   31
三塊厝    786111   933   16   824381  1013   15
三姓橋   1760760  2125    3  1594509  1603   16
三民      12535    52    0    18652    57    0
...       ...   ...  ...      ...   ...  ...
鼓山     469414  2352    9   476151  1055   16
龍井     596686  2131   36   589722  5506   35
龍泉      24965   103    0    30795   390    1
龍港      30146   106    2    38335   130    2
龜山     141198   389    3   134504   287    0

[241 rows x 6 columns]

In [45]:
station_array = station_all_df1.values
station_array[:,0]

array([Timestamp('2020-01-01 00:00:00'), Timestamp('2020-01-01 00:00:00'),
       Timestamp('2020-01-01 00:00:00'), ...,
       Timestamp('2022-12-31 00:00:00'), Timestamp('2022-12-31 00:00:00'),
       Timestamp('2022-12-31 00:00:00')], dtype=object)

In [46]:
station_array[:,[0,2,3]]

array([[Timestamp('2020-01-01 00:00:00'), 7552, 8154],
       [Timestamp('2020-01-01 00:00:00'), 1020, 1135],
       [Timestamp('2020-01-01 00:00:00'), 1623, 1972],
       ...,
       [Timestamp('2022-12-31 00:00:00'), 131, 159],
       [Timestamp('2022-12-31 00:00:00'), 494, 426],
       [Timestamp('2022-12-31 00:00:00'), 313, 274]], dtype=object)

In [47]:
station_all_df1.query('乘車日=="2023"')

/tmp/ipykernel_54736/2553819135.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  station_all_df1.query('乘車日=="2023"')


,乘車日,車站代碼,進站人數,出站人數,中文站名
147061,2023-01-01,900,8030,8241,基隆
147062,2023-01-01,910,1017,1326,三坑
147063,2023-01-01,920,1400,1705,八堵
147064,2023-01-01,930,3558,4398,七堵
147065,2023-01-01,940,1509,1815,百福
...,...,...,...,...,...
147294,2023-01-01,7360,4543,5222,瑞芳
147295,2023-01-01,7361,93,123,海科館
147296,2023-01-01,7362,415,429,八斗子
147297,2023-01-01,7380,485,495,四腳亭


In [48]:

station_all_df1['乘車日'].dt.strftime('%Y-%m')

0         2020-01
1         2020-01
2         2020-01
3         2020-01
4         2020-01
           ...   
406756    2022-12
406757    2022-12
406758    2022-12
406759    2022-12
406760    2022-12
Name: 乘車日, Length: 406761, dtype: object

In [52]:
df3 = station_all_df1.groupby(['中文站名',station_all_df1['乘車日'].dt.year])[['進站人數','出站人數']].agg(['sum','max','min'])
df3

進站人數                 出站人數            
               sum   max   min      sum   max   min
中文站名 乘車日                                           
七堵   2019  1512951  7417  2103  1560506  8078  2438
     2020  2023475  7366  2616  2055442  8288  2607
     2021  1619233  7314   614  1594220  7702   630
     2022  1764417  7280   141  1776557  7289   210
     2023  2124487  7704  1472  2128604  7754  1459
...            ...   ...   ...      ...   ...   ...
龜山   2019    26408   199    49    25265   166    48
     2020    33665   209    43    32326   206    30
     2021    23193   194     9    22534   157     9
     2022    27966   389     3    24766   178     0
     2023    29966   255    31    29613   287    33

[1198 rows x 6 columns]

In [53]:
df3.columns.names = ['人數','統計']
df3

人數            進站人數                 出站人數            
統計             sum   max   min      sum   max   min
中文站名 乘車日                                           
七堵   2019  1512951  7417  2103  1560506  8078  2438
     2020  2023475  7366  2616  2055442  8288  2607
     2021  1619233  7314   614  1594220  7702   630
     2022  1764417  7280   141  1776557  7289   210
     2023  2124487  7704  1472  2128604  7754  1459
...            ...   ...   ...      ...   ...   ...
龜山   2019    26408   199    49    25265   166    48
     2020    33665   209    43    32326   206    30
     2021    23193   194     9    22534   157     9
     2022    27966   389     3    24766   178     0
     2023    29966   255    31    29613   287    33

[1198 rows x 6 columns]

In [56]:
station_all_df1.groupby(['中文站名',station_all_df1['乘車日'].dt.strftime('%Y-%m')])[['進站人數','出站人數']].agg(['sum','max','min'])

進站人數                出站人數            
                 sum   max   min     sum   max   min
中文站名 乘車日                                            
七堵   2019-04   46711  6635  4616   47817  7042  4552
     2019-05  186400  7075  4599  190985  7568  4533
     2019-06  173853  7141  4150  177242  7319  4242
     2019-07  182569  6791  4433  190198  7482  4652
     2019-08  172647  6898  2377  178678  7202  2500
...              ...   ...   ...     ...   ...   ...
龜山   2023-08    2318   121    45    2378   117    50
     2023-09    2398   126    46    2393   135    42
     2023-10    2660   128    31    2669   160    38
     2023-11    2563   110    48    2594   137    49
     2023-12    3091   255    48    2794   287    48

[13619 rows x 6 columns]

In [55]:
df3.stack(level=['人數','統計']).unstack(level=['中文站名','統計'])

/tmp/ipykernel_54736/1981058358.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df3.stack(level=['人數','統計']).unstack(level=['中文站名','統計'])


中文站名              七堵                        三坑                      三塊厝  \
統計               sum     max     min       sum     max    min       sum   
乘車日  人數                                                                   
2019 出站人數  1560506.0  8078.0  2438.0  347841.0  1778.0  499.0  111863.0   
     進站人數  1512951.0  7417.0  2103.0  342775.0  1606.0  426.0  105842.0   
2020 出站人數  2055442.0  8288.0  2607.0  466250.0  1645.0  676.0  160297.0   
     進站人數  2023475.0  7366.0  2616.0  457042.0  1557.0  696.0  147363.0   
2021 出站人數  1594220.0  7702.0   630.0  383427.0  1664.0  182.0  155794.0   
     進站人數  1619233.0  7314.0   614.0  372260.0  1551.0  198.0  149138.0   
2022 出站人數  1776557.0  7289.0   210.0  422225.0  1618.0   31.0  178429.0   
     進站人數  1764417.0  7280.0   141.0  411286.0  1648.0   34.0  173337.0   
2023 出站人數  2128604.0  7754.0  1459.0  521151.0  1835.0  404.0  217998.0   
     進站人數  2124487.0  7704.0  1472.0  497154.0  1715.0  347.0  210431.0   

中文站名                           三姓橋  ...     龍井      龍泉                  龍港  \
統計            max    min       sum  ...    min     sum    max  min     sum   
乘車日  人數                             ...                                      
2019 出站人數   720.0  108.0  260777.0  ...  240.0  4960.0  303.0  2.0  5417.0   
     進站人數   777.0   79.0  288098.0  ...  249.0  4115.0   76.0  1.0  4413.0   
2020 出站人數   807.0  184.0  345666.0  ...  161.0  6710.0  274.0  1.0  7490.0   
     進站人數   794.0  186.0  386310.0  ...  201.0  5827.0   84.0  3.0  5581.0   
2021 出站人數   835.0   43.0  276783.0  ...   35.0  4841.0   66.0  1.0  7578.0   
     進站人數   760.0   48.0  307925.0  ...   36.0  4542.0  103.0  0.0  5394.0   
2022 出站人數   877.0   15.0  312076.0  ...  117.0  6038.0  390.0  1.0  8217.0   
     進站人數   811.0   16.0  345589.0  ...  115.0  4218.0   58.0  0.0  6096.0   
2023 出站人數  1013.0  125.0  399207.0  ...  114.0  8246.0  271.0  1.0  9633.0   
     進站人數   933.0  121.0  432838.0  ...  126.0  6263.0   92.0  0.0  8662.0   

中文站名                        龜山               
統計           max  min      sum    max   min  
乘車日  人數                                      
2019 出站人數   45.0  7.0  25265.0  166.0  48.0  
     進站人數   45.0  5.0  26408.0  199.0  49.0  
2020 出站人數   96.0  2.0  32326.0  206.0  30.0  
     進站人數   88.0  4.0  33665.0  209.0  43.0  
2021 出站人數   81.0  3.0  22534.0  157.0   9.0  
     進站人數   96.0  2.0  23193.0  194.0   9.0  
2022 出站人數  112.0  7.0  24766.0  178.0   0.0  
     進站人數   44.0  6.0  27966.0  389.0   3.0  
2023 出站人數  130.0  7.0  29613.0  287.0  33.0  
     進站人數  106.0  8.0  29966.0  255.0  31.0  

[10 rows x 723 columns]